# CSGO Match Prediction with Logistic Regression

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
# Ideas to try
# ============
# - higher learning rate, higher c, with one hot encoding
# - polyfit with no one-hot
# - polyfit with one hot
# - psuedo one-hot encoding vs summing winner data

## Loading Data

In [3]:
df = pd.read_csv('./economy.csv', low_memory=False)

In [4]:
winners = df.T.apply(pd.Series.last_valid_index)

In [5]:
winners_col = []
winnerarr = np.array(winners)
for i in range(df.shape[0]):
    winners_col.append(df[winnerarr[i]][i])
    #winners_col.append(df["team_" + str(int(df[winnerarr[i]][i]))][i])

In [6]:
df['match_winner'] = winners_col

In [7]:
round_win_col_names = [
 '16_winner',
 '17_winner',
 '18_winner',
 '19_winner',
 '20_winner',
 '21_winner',
 '22_winner',
 '23_winner',
 '24_winner',
 '25_winner',
 '26_winner',
 '27_winner',
 '28_winner',
 '29_winner',
 '30_winner']

df = df.drop(columns=round_win_col_names)
df = df.drop(columns=["best_of", "date", "t2_start"])
df = df.drop(columns=["match_id", "event_id"])

droprounds = ['16_t1', '17_t1', '18_t1', '19_t1', '20_t1', '21_t1', '22_t1', '23_t1', '24_t1', '25_t1', '26_t1', '27_t1', '28_t1', '29_t1', '30_t1', '16_t2', '17_t2', '18_t2', '19_t2', '20_t2', '21_t2', '22_t2', '23_t2', '24_t2', '25_t2', '26_t2', '27_t2', '28_t2', '29_t2', '30_t2']
df = df.drop(columns=droprounds)

In [8]:
colnames = list(df.columns)

In [9]:
#print(colnames)
xnames = colnames[:len(colnames) - 1]
#print(xnames)
X = df[xnames]

set_of_teams = set(list(X['team_1']) + list(X['team_2']))
team_to_num = dict(zip(set_of_teams, range(len(set_of_teams))))

#Replace team's name with their number
X['team_1'] = X['team_1'].replace(team_to_num)
X['team_2'] = X['team_2'].replace(team_to_num)
y = df["match_winner"]

#Convert from t1_start to t1_t and convert from t,ct to binary
X['t1_t'] = X['t1_start'].apply(lambda x: 1 if x == 't' else 0)
X = X.drop('t1_start', axis=1)

#Replace map names with numbers
map_to_num = dict(zip(X['_map'].unique(), range(len(X['_map'].unique()))))
X['_map'] = X['_map'].replace(map_to_num)

X

<ipython-input-9-072520d650ff>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['team_1'] = X['team_1'].replace(team_to_num)
<ipython-input-9-072520d650ff>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['team_2'] = X['team_2'].replace(team_to_num)


,team_1,team_2,_map,1_t1,2_t1,3_t1,4_t1,5_t1,6_t1,7_t1,...,7_winner,8_winner,9_winner,10_winner,11_winner,12_winner,13_winner,14_winner,15_winner,t1_t
0,1551,1846,0,4350.0,1100.0,22100.0,9350.0,25750.0,10400.0,24600.0,...,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1
1,1551,1846,1,3900.0,7400.0,23250.0,28500.0,31900.0,31700.0,18950.0,...,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,0
2,1551,1846,2,4150.0,14300.0,2000.0,24800.0,9000.0,23150.0,21850.0,...,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1
3,1846,632,1,4150.0,18050.0,21000.0,25850.0,25000.0,25000.0,27250.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
4,1846,632,0,4200.0,10000.0,22000.0,24500.0,27550.0,29350.0,31950.0,...,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43229,1308,851,9,4250.0,7400.0,17200.0,18500.0,26750.0,27450.0,26950.0,...,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1
43230,632,893,4,4250.0,1600.0,24250.0,17400.0,22050.0,6600.0,25650.0,...,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1
43231,302,2213,2,4350.0,7650.0,20000.0,21500.0,29150.0,26900.0,8750.0,...,2.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,0
43232,1189,940,2,4150.0,11800.0,1200.0,22950.0,25750.0,26850.0,27850.0,...,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1


In [10]:
y

0        2.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
43229    2.0
43230    1.0
43231    1.0
43232    1.0
43233    1.0
Name: match_winner, Length: 43234, dtype: float64

***

## One-Hot Conversion (Encoding Teams)

In [11]:
# # Convert the team names back into strings again
# num_to_team = dict(zip(range(len(set_of_teams)), set_of_teams))
# all_teams = [num_to_team[i] for i in range(len(set_of_teams))]

# # Extract the team data from the original DataFrame
# team_1_arr = np.array(X["team_1"])
# team_2_arr = np.array(X["team_2"])

# # Create the one-hot vectors for each row of the data, one for each team
# one_hot_t1 = np.zeros((team_1_arr.size, len(num_to_team)))
# one_hot_t1[np.arange(team_1_arr.size), team_1_arr] = 1

# one_hot_t2 = np.zeros((team_2_arr.size, len(num_to_team)))
# one_hot_t2[np.arange(team_2_arr.size), team_2_arr] = 1

# # Combine the matrix of one-hot vectors for team_1 and team_2 together to create a single matrix
# one_hot = np.hstack((one_hot_t1, one_hot_t2))

# # Turn the numpy array into a pandas DataFrame and add it to the dataset
# one_hot_df = pd.DataFrame(one_hot, columns = ["team_1_%s" %(i) for i in all_teams] + ["team_2_%s" %(i) for i in all_teams])
# X = X.drop(columns = ["team_1", "team_2"])
# X = X.join(one_hot_df)

***

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

scaler = StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## LogReg w/ No Regularization

In [13]:
logreg_ridge_none = linear_model.LogisticRegression(penalty='none') # L2 regularization

In [14]:
logreg_ridge_none.fit(X_train, y_train)

LogisticRegression(penalty='none')

In [15]:
yhat = logreg_ridge_none.predict(X_test)# the predict method will return 0 or 1
acc = np.mean(yhat == y_test)
print("Accuracy on test data = %f" % acc)

Accuracy on test data = 0.775280


In [16]:
print("The regularization parameter:", logreg_ridge_none.C)
print('Accuracy on the test data is {0:f}'.format(acc))
W_l1 = logreg_ridge_none.coef_


data = {'feature': xnames, 'slope': np.squeeze(W_l1)}
dfslope = pd.DataFrame(data=data)
dfslope

The regularization parameter: 1.0
Accuracy on the test data is 0.775280


,feature,slope
0,team_1,0.030361
1,team_2,-0.032039
2,_map,-0.008675
3,t1_start,0.029266
4,1_t1,-0.052910
5,2_t1,-0.030457
6,3_t1,-0.047672
7,4_t1,0.005822
8,5_t1,0.002263
9,6_t1,0.021636


In [17]:
# For the test data we can see the number of tpr, fpr, fnr, and tnr in the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(yhat, y_test)

array([[4603, 1240],
       [1189, 3777]], dtype=int64)

In [18]:
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
cv = KFold(n_splits=10, random_state=1, shuffle=True)
scores = cross_val_score(logreg_ridge_none, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

Accuracy: 0.749 (0.007)


In [19]:
from sklearn.metrics import precision_recall_fscore_support
# Find Precision, recall and fscore using precision_recall_fscore_support nethod of sklearn
# Using y_train and y_hat_logreg
prec,recal,fscore,_= precision_recall_fscore_support(y_test,yhat,average='binary')
print('prec: ', prec)
print('recal: ', recal)
print('fscore: ', fscore)

prec:  0.7877802498716413
recal:  0.7947168508287292
fscore:  0.7912333476579286


## L1 Regularization

In [20]:
logreg_ridge_l1 = linear_model.LogisticRegression(solver='liblinear', penalty='l1',warm_start=True, C = 0.01) # L2 regularization

In [21]:
logreg_ridge_l1.fit(X_train, y_train)

LogisticRegression(C=0.01, penalty='l1', solver='liblinear', warm_start=True)

In [22]:
yhat_l1 = logreg_ridge_l1.predict(X_test)# the predict method will return 0 or 1
acc = np.mean(yhat_l1 == y_test)
print("Accuracy on test data = %f" % acc)

Accuracy on test data = 0.776020


In [23]:
print("The regularization parameter:", logreg_ridge_l1.C)
print('Accuracy on the test data is {0:f}'.format(acc))
W_l1 = logreg_ridge_l1.coef_


data = {'feature': xnames, 'slope': np.squeeze(W_l1)}
dfslope = pd.DataFrame(data=data)
dfslope

The regularization parameter: 0.01
Accuracy on the test data is 0.776020


,feature,slope
0,team_1,0.010799
1,team_2,-0.011485
2,_map,0.000000
3,t1_start,0.012844
4,1_t1,-0.029293
5,2_t1,0.000000
6,3_t1,-0.009554
7,4_t1,0.000000
8,5_t1,0.000000
9,6_t1,0.000000


In [24]:
# For the test data we can see the number of tpr, fpr, fnr, and tnr in the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(yhat_l1, y_test)

array([[4548, 1177],
       [1244, 3840]], dtype=int64)

In [25]:
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
cv = KFold(n_splits=10, random_state=1, shuffle=True)
scores = cross_val_score(logreg_ridge_l1, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

Accuracy: 0.770 (0.009)


In [26]:
from sklearn.metrics import precision_recall_fscore_support
# Find Precision, recall and fscore using precision_recall_fscore_support nethod of sklearn
# Using y_train and y_hat_logreg
prec,recal,fscore,_= precision_recall_fscore_support(y_test,yhat_l1,average='binary')
print('prec: ', prec)
print('recal: ', recal)
print('fscore: ', fscore)

prec:  0.794410480349345
recal:  0.7852209944751382
fscore:  0.7897890075540505


## L2 Regularization

In [27]:
logreg_ridge_l2 = linear_model.LogisticRegression(penalty='l2', C = 0.01) # L2 regularization

In [28]:
logreg_ridge_l2.fit(X_train, y_train)

LogisticRegression(C=0.01)

In [29]:
yhat_l2 = logreg_ridge_l2.predict(X_test)# the predict method will return 0 or 1
acc = np.mean(yhat_l2 == y_test)
print("Accuracy on test data = %f" % acc)

Accuracy on test data = 0.775465


In [30]:
print("The regularization parameter:", logreg_ridge_l2.C)
print('Accuracy on the test data is {0:f}'.format(acc))
W_l1 = logreg_ridge_l2.coef_


data = {'feature': xnames, 'slope': np.squeeze(W_l1)}
dfslope = pd.DataFrame(data=data)
dfslope

The regularization parameter: 0.01
Accuracy on the test data is 0.775465


,feature,slope
0,team_1,0.030150
1,team_2,-0.031222
2,_map,-0.008007
3,t1_start,0.028744
4,1_t1,-0.075517
5,2_t1,-0.047147
6,3_t1,-0.053042
7,4_t1,-0.002222
8,5_t1,-0.007412
9,6_t1,0.012262


In [31]:
# For the test data we can see the number of tpr, fpr, fnr, and tnr in the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(yhat_l2, y_test)

array([[4612, 1247],
       [1180, 3770]], dtype=int64)

In [32]:
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
cv = KFold(n_splits=10, random_state=1, shuffle=True)
scores = cross_val_score(logreg_ridge_l2, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

Accuracy: 0.749 (0.007)


In [33]:
from sklearn.metrics import precision_recall_fscore_support
# Find Precision, recall and fscore using precision_recall_fscore_support nethod of sklearn
# Using y_train and y_hat_logreg
prec,recal,fscore,_= precision_recall_fscore_support(y_test,yhat_l2,average='binary')
print('prec: ', prec)
print('recal: ', recal)
print('fscore: ', fscore)

prec:  0.7871650452295613
recal:  0.7962707182320442
fscore:  0.7916917002832373


***

## Polynomial Transformation Fit

In [48]:
# Import useful polynomial library
import numpy.polynomial.polynomial as poly

## Loading Data

In [49]:
df = pd.read_csv('./economy.csv', low_memory=False)

In [50]:
winners = df.T.apply(pd.Series.last_valid_index)

In [51]:
winners_col = []
winnerarr = np.array(winners)
for i in range(df.shape[0]):
    winners_col.append(df[winnerarr[i]][i])
    #winners_col.append(df["team_" + str(int(df[winnerarr[i]][i]))][i])

In [52]:
df['match_winner'] = winners_col

In [53]:
round_win_col_names = [
 '16_winner',
 '17_winner',
 '18_winner',
 '19_winner',
 '20_winner',
 '21_winner',
 '22_winner',
 '23_winner',
 '24_winner',
 '25_winner',
 '26_winner',
 '27_winner',
 '28_winner',
 '29_winner',
 '30_winner']

df = df.drop(columns=round_win_col_names)
df = df.drop(columns=["best_of", "date", "t2_start"])
df = df.drop(columns=["match_id", "event_id"])

droprounds = ['16_t1', '17_t1', '18_t1', '19_t1', '20_t1', '21_t1', '22_t1', '23_t1', '24_t1', '25_t1', '26_t1', '27_t1', '28_t1', '29_t1', '30_t1', '16_t2', '17_t2', '18_t2', '19_t2', '20_t2', '21_t2', '22_t2', '23_t2', '24_t2', '25_t2', '26_t2', '27_t2', '28_t2', '29_t2', '30_t2']
df = df.drop(columns=droprounds)
df = df.sample(frac = .55)

In [54]:
colnames = list(df.columns)

In [55]:
#print(colnames)
xnames = colnames[:len(colnames) - 1]
#print(xnames)
X = df[xnames]

set_of_teams = set(list(X['team_1']) + list(X['team_2']))
team_to_num = dict(zip(set_of_teams, range(len(set_of_teams))))

#Replace team's name with their number
X['team_1'] = X['team_1'].replace(team_to_num)
X['team_2'] = X['team_2'].replace(team_to_num)
y = df["match_winner"]

#Convert from t1_start to t1_t and convert from t,ct to binary
X['t1_t'] = X['t1_start'].apply(lambda x: 1 if x == 't' else 0)
X = X.drop('t1_start', axis=1)

#Replace map names with numbers
map_to_num = dict(zip(X['_map'].unique(), range(len(X['_map'].unique()))))
X['_map'] = X['_map'].replace(map_to_num)

<ipython-input-55-9df4de8fea1f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['team_1'] = X['team_1'].replace(team_to_num)
<ipython-input-55-9df4de8fea1f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['team_2'] = X['team_2'].replace(team_to_num)


In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

scaler = StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
import gc
rmses = []
degrees = np.arange(2, 10)
min_rmse, min_deg = 1e10, 0

for deg in degrees:

    # Train features
    poly_features = PolynomialFeatures(degree=deg, include_bias=False)
    x_poly_train = poly_features.fit_transform(X_train)

    # Logistic regression
    poly_reg = linear_model.LogisticRegression(penalty='l2', C = 0.01)
    poly_reg.fit(x_poly_train, y_train)

    # Compare with test data
    x_poly_test = poly_features.fit_transform(X_test)
    print("Fit Deg: ", deg)
    poly_predict = poly_reg.predict(x_poly_test)
    print("Predict Deg: ", deg)
    poly_mse = mean_squared_error(y_test, poly_predict)
    poly_rmse = np.sqrt(poly_mse)
    rmses.append(poly_rmse)
    
    # Cross-validation of degree
    if min_rmse > poly_rmse:
        min_rmse = poly_rmse
        min_deg = deg
    
    del poly_features
    del x_poly_train
    del poly_reg
    del x_poly_test
    del poly_predict
    gc.collect()

# Plot and present results
print('Best degree {} with RMSE {}'.format(min_deg, min_rmse))
        
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(degrees, rmses)
ax.set_yscale('log')
ax.set_xlabel('Degree')
ax.set_ylabel('RMSE')

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
# create transform
trans = PolynomialFeatures(degree=dval[imin])
# fit and transform
X = trans.fit_transform(X)
print('Degree: %d, Features: %d' % (dval[imin], X.shape[1]))

***